# Baseline Models: null_model() and naive_reg()

This notebook demonstrates **baseline models** in py-tidymodels, which are essential for benchmarking any modeling project.

## Why Baselines are Critical

**Rule #1 of Machine Learning:** If your model can't beat a simple baseline, it's not adding value.

Baseline models provide:
- **Reality check**: Is your complex model actually learning something useful?
- **Minimum bar**: The simplest possible prediction strategy
- **Quick implementation**: Minutes to fit, not hours
- **Interpretability**: Everyone understands "predict the mean" or "use last value"

## Models Covered

1. **null_model()** - Predicts a constant (mean, median, or mode)
   - Regression: mean or median of training target
   - Classification: most frequent class
   - Use case: General baseline for any problem

2. **naive_reg()** - Time series naive forecasting methods
   - **naive**: Last observed value (random walk)
   - **seasonal_naive**: Last value from same season
   - **drift**: Linear trend extrapolation
   - Use case: Essential baseline for time series forecasting

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Import py-tidymodels packages
from py_parsnip import null_model, naive_reg, linear_reg, rand_forest
from py_recipes import recipe, step_date, step_lag, step_normalize
from py_workflows import workflow
from py_rsample import initial_time_split
from py_visualize import plot_model_comparison

print("✓ All packages imported successfully")

## Part 1: null_model() - The Simplest Baseline

The null model is the most basic baseline: predict a constant for all observations.

### For Regression:
- Predicts the **mean** (or median) of the training target
- Represents "what if I just guessed the average every time?"
- RMSE of null model = standard deviation of target

### For Classification:
- Predicts the **mode** (most frequent class)
- Represents "what if I just guessed the majority class every time?"

### Setup: Create Sample Data

In [ ]:
# Create simple regression data
np.random.seed(42)
n = 200

# Features
x1 = np.random.randn(n)
x2 = np.random.randn(n)
x3 = np.random.randn(n)

# Target: linear relationship + noise
y = 10 + 2 * x1 + 1.5 * x2 - 0.8 * x3 + np.random.randn(n) * 2

data = pd.DataFrame({
    'x1': x1,
    'x2': x2,
    'x3': x3,
    'y': y
})

# Train/test split
train_data = data.iloc[:150]
test_data = data.iloc[150:]

print(f"Training observations: {len(train_data)}")
print(f"Testing observations: {len(test_data)}")
print(f"\nTarget statistics (training):")
print(f"  Mean: {train_data['y'].mean():.4f}")
print(f"  Std:  {train_data['y'].std():.4f}")
print(f"  Min:  {train_data['y'].min():.4f}")
print(f"  Max:  {train_data['y'].max():.4f}")

### Fit null_model() - Predicts Mean

In [ ]:
# Create null model specification
spec_null = null_model()

print("Null Model Specification:")
print(spec_null)
print("\nThis model will predict the mean of the training target for all observations.")

In [ ]:
# Fit null model (no features needed, but we'll use the formula for consistency)
fit_null = spec_null.fit(formula='y ~ 1', data=train_data)

# Generate predictions
pred_null = fit_null.predict(test_data)

print("✓ Null model fitted")
print(f"\nPredicted value (constant): {pred_null['.pred'].iloc[0]:.4f}")
print(f"Training mean:              {train_data['y'].mean():.4f}")
print("\nNote: All predictions are identical (the training mean)")

### Compare to Linear Regression and Random Forest

In [ ]:
# Fit linear regression
spec_linear = linear_reg()
fit_linear = spec_linear.fit(formula='y ~ x1 + x2 + x3', data=train_data)
pred_linear = fit_linear.predict(test_data)

print("✓ Linear regression fitted")

In [ ]:
# Fit random forest
spec_rf = rand_forest(trees=100, mode='regression')
fit_rf = spec_rf.fit(formula='y ~ x1 + x2 + x3', data=train_data)
pred_rf = fit_rf.predict(test_data)

print("✓ Random forest fitted")

### Extract Metrics and Compare

In [ ]:
# Extract stats DataFrames
_, _, stats_null = fit_null.extract_outputs()
_, _, stats_linear = fit_linear.extract_outputs()
_, _, stats_rf = fit_rf.extract_outputs()

print("Null Model Performance (Test Set):")
print("=" * 50)
test_stats_null = stats_null[stats_null['split'] == 'test']
for _, row in test_stats_null.iterrows():
    print(f"{row['metric']:12s}: {row['value']:.6f}")

In [ ]:
# Visual comparison
fig = plot_model_comparison(
    stats_list=[stats_null, stats_linear, stats_rf],
    model_names=["Null Model (Mean)", "Linear Regression", "Random Forest"],
    metrics=["rmse", "mae", "r_squared"],
    split="test",
    plot_type="bar",
    title="Model Performance vs Null Baseline",
    height=500
)

fig.show()

print("\n📊 Interpretation:")
print("  • Null model: RMSE ≈ Std(y) - no learning, just predicts mean")
print("  • Linear & RF: RMSE < Std(y) - they've learned something!")
print("  • R² = 0 for null model (baseline)")
print("  • If R² < 0, your model is WORSE than null model!")

### Key Insights: null_model()

1. **RMSE of null model ≈ Standard deviation of target**
   - This is the "no learning" baseline
   - Any model with RMSE > σ(y) is worse than predicting the mean

2. **R² is defined relative to null model**
   - R² = 1 - (SSE / SST) where SST = variance explained by null model
   - R² = 0 means you're as good as null model
   - R² < 0 means you're WORSE than null model

3. **Use cases:**
   - First baseline for ANY regression problem
   - Sanity check before complex modeling
   - Quick benchmark for model selection

## Part 2: naive_reg() - Time Series Baselines

For time series forecasting, we need different baselines that account for temporal structure.

### Three Naive Methods:

1. **Naive (Random Walk)**
   - Formula: ŷ_t = y_{t-1}
   - Prediction: "Tomorrow will be like today"
   - Best for: Random walks, stock prices

2. **Seasonal Naive**
   - Formula: ŷ_t = y_{t-seasonal_period}
   - Prediction: "This Monday will be like last Monday"
   - Best for: Strong seasonal patterns

3. **Drift Method**
   - Formula: ŷ_t = y_{t-1} + (y_T - y_1) / (T - 1)
   - Prediction: "Continue the linear trend"
   - Best for: Data with consistent trend

### Setup: Create Time Series Data

In [ ]:
# Create time series with trend + seasonality
np.random.seed(42)
dates = pd.date_range('2020-01-01', periods=365, freq='D')
time_index = np.arange(len(dates))

# Components
trend = time_index * 0.3  # Upward trend
weekly_season = 10 * np.sin(2 * np.pi * time_index / 7)  # Weekly pattern
noise = np.random.randn(len(dates)) * 3

y = trend + weekly_season + noise + 100

ts_data = pd.DataFrame({
    'date': dates,
    'value': y
})

# Split: 80% train, 20% test
split = initial_time_split(ts_data, prop=0.8)
train_ts = split.training()
test_ts = split.testing()

print(f"Training: {len(train_ts)} observations ({train_ts['date'].min()} to {train_ts['date'].max()})")
print(f"Testing:  {len(test_ts)} observations ({test_ts['date'].min()} to {test_ts['date'].max()})")
print(f"\nData characteristics:")
print(f"  - Trend: +0.3 per day")
print(f"  - Seasonality: Weekly (period=7)")
print(f"  - Noise: σ=3")

### Method 1: Naive (Last Value)

In [ ]:
# Create naive model (method="naive" is default)
spec_naive = naive_reg(method="naive")

print("Naive Model (Random Walk):")
print(spec_naive)
print("\nPrediction strategy: ŷ_t = y_{t-1}")
print("Use case: Random walks, efficient markets")

In [ ]:
# Fit naive model
fit_naive = spec_naive.fit(formula='value ~ date', data=train_ts)
pred_naive = fit_naive.predict(test_ts)

print("✓ Naive model fitted")
print(f"\nLast training value: {train_ts['value'].iloc[-1]:.4f}")
print(f"First prediction:    {pred_naive['.pred'].iloc[0]:.4f}")
print("\nNote: First prediction = last training value (random walk)")

### Method 2: Seasonal Naive (Last Seasonal Value)

In [ ]:
# Create seasonal naive model (weekly seasonality = 7 days)
spec_snaive = naive_reg(seasonal_period=7, method="seasonal_naive")

print("Seasonal Naive Model:")
print(spec_snaive)
print("\nPrediction strategy: ŷ_t = y_{t-7}")
print("Use case: Strong seasonal patterns (e.g., Monday like last Monday)")

In [ ]:
# Fit seasonal naive model
fit_snaive = spec_snaive.fit(formula='value ~ date', data=train_ts)
pred_snaive = fit_snaive.predict(test_ts)

print("✓ Seasonal naive model fitted")
print("\nPrediction logic:")
print("  - First test date (Thursday): uses last Thursday's value")
print("  - Each day: uses same weekday from 7 days ago")

### Method 3: Drift (Linear Trend)

In [ ]:
# Create drift model
spec_drift = naive_reg(method="drift")

print("Drift Model:")
print(spec_drift)
print("\nPrediction strategy: ŷ_t = y_{t-1} + (y_T - y_1) / (T - 1)")
print("Use case: Data with consistent linear trend")

In [ ]:
# Fit drift model
fit_drift = spec_drift.fit(formula='value ~ date', data=train_ts)
pred_drift = fit_drift.predict(test_ts)

# Calculate drift rate
first_val = train_ts['value'].iloc[0]
last_val = train_ts['value'].iloc[-1]
n_obs = len(train_ts)
drift_rate = (last_val - first_val) / (n_obs - 1)

print("✓ Drift model fitted")
print(f"\nDrift rate: {drift_rate:.4f} per day")
print(f"First training value: {first_val:.4f}")
print(f"Last training value:  {last_val:.4f}")
print(f"\nPredictions extrapolate this linear trend forward")

### Compare All Three Naive Methods

In [ ]:
# Extract stats
_, _, stats_naive = fit_naive.extract_outputs()
_, _, stats_snaive = fit_snaive.extract_outputs()
_, _, stats_drift = fit_drift.extract_outputs()

# Visual comparison
fig = plot_model_comparison(
    stats_list=[stats_naive, stats_snaive, stats_drift],
    model_names=["Naive (Last Value)", "Seasonal Naive (Weekly)", "Drift (Trend)"],
    metrics=["rmse", "mae", "r_squared"],
    split="test",
    plot_type="bar",
    title="Naive Methods Comparison",
    height=500
)

fig.show()

print("\n📊 Which naive method performs best?")
test_rmses = {
    "Naive": stats_naive[stats_naive['metric'] == 'rmse']['value'].values[1],
    "Seasonal Naive": stats_snaive[stats_snaive['metric'] == 'rmse']['value'].values[1],
    "Drift": stats_drift[stats_drift['metric'] == 'rmse']['value'].values[1]
}

best_method = min(test_rmses, key=test_rmses.get)
print(f"\nBest naive method: {best_method}")
print(f"  RMSE: {test_rmses[best_method]:.4f}")
print("\nExpected: Seasonal Naive should win (data has weekly seasonality)")

### Compare Naive Methods to ML Models

In [ ]:
# Build ML models with feature engineering
rec = (
    recipe(value ~ date, data=train_ts)
    .step_date('date', features=['dow', 'week'])
    .step_lag('value', lags=[1, 7, 14])
    .step_normalize(['value_lag_1', 'value_lag_7', 'value_lag_14'])
)

# Linear regression with features
wf_linear = (
    workflow()
    .add_recipe(rec)
    .add_model(linear_reg())
)
fit_linear_ts = wf_linear.fit(train_ts)

# Random forest with features
wf_rf = (
    workflow()
    .add_recipe(rec)
    .add_model(rand_forest(trees=100, mode='regression'))
)
fit_rf_ts = wf_rf.fit(train_ts)

print("✓ ML models fitted with date features and lags")

In [ ]:
# Extract stats
_, _, stats_linear_ts = fit_linear_ts.extract_outputs()
_, _, stats_rf_ts = fit_rf_ts.extract_outputs()

# Compare all models
fig = plot_model_comparison(
    stats_list=[
        stats_naive, stats_snaive, stats_drift,
        stats_linear_ts, stats_rf_ts
    ],
    model_names=[
        "Naive", "Seasonal Naive", "Drift",
        "Linear Regression", "Random Forest"
    ],
    metrics=["rmse", "mae", "r_squared"],
    split="test",
    plot_type="bar",
    title="Naive Baselines vs ML Models",
    height=500,
    width=900
)

fig.show()

print("\n📊 Key Question: Do ML models beat the baselines?")
print("\nTest RMSE:")
all_rmses = {
    "Naive": stats_naive[stats_naive['metric'] == 'rmse']['value'].values[1],
    "Seasonal Naive": stats_snaive[stats_snaive['metric'] == 'rmse']['value'].values[1],
    "Drift": stats_drift[stats_drift['metric'] == 'rmse']['value'].values[1],
    "Linear Regression": stats_linear_ts[stats_linear_ts['metric'] == 'rmse']['value'].values[1],
    "Random Forest": stats_rf_ts[stats_rf_ts['metric'] == 'rmse']['value'].values[1]
}

for model, rmse in sorted(all_rmses.items(), key=lambda x: x[1]):
    print(f"  {model:20s}: {rmse:.4f}")

best_overall = min(all_rmses, key=all_rmses.get)
print(f"\n🏆 Best model: {best_overall}")

# Check if ML beat best naive
best_naive_rmse = min([test_rmses[k] for k in test_rmses.keys()])
ml_beat_naive = min(all_rmses['Linear Regression'], all_rmses['Random Forest']) < best_naive_rmse

if ml_beat_naive:
    print("\n✓ ML models beat naive baselines - they're adding value!")
else:
    print("\n⚠️  ML models didn't beat naive baselines - reconsider your approach!")

### When to Use Each Naive Method

| Method | Best For | Example Use Cases |
|--------|----------|------------------|
| **Naive** | Random walks, no pattern | Stock prices, exchange rates |
| **Seasonal Naive** | Strong seasonality | Retail sales, website traffic, temperature |
| **Drift** | Consistent linear trend | Population growth, cumulative metrics |

### Rule of Thumb:
1. **Always** try all three naive methods
2. The best naive method is your **minimum acceptable baseline**
3. If your ML model can't beat it, **don't deploy the ML model**

## Part 3: Benchmarking Workflow

Here's a recommended workflow for any modeling project:

In [ ]:
# Step 1: Fit baseline(s)
print("Step 1: Establish Baseline")
print("=" * 50)

# For regression: null_model
baseline = null_model()
fit_baseline = baseline.fit(formula='y ~ 1', data=train_data)
_, _, stats_baseline = fit_baseline.extract_outputs()
baseline_rmse = stats_baseline[stats_baseline['metric'] == 'rmse']['value'].values[1]

print(f"Baseline RMSE: {baseline_rmse:.4f}")
print("This is the minimum bar. Any model must beat this.\n")

In [ ]:
# Step 2: Try simple model
print("Step 2: Try Simple Model (Linear Regression)")
print("=" * 50)

simple_model = linear_reg()
fit_simple = simple_model.fit(formula='y ~ x1 + x2 + x3', data=train_data)
_, _, stats_simple = fit_simple.extract_outputs()
simple_rmse = stats_simple[stats_simple['metric'] == 'rmse']['value'].values[1]

print(f"Simple Model RMSE: {simple_rmse:.4f}")

improvement_pct = ((baseline_rmse - simple_rmse) / baseline_rmse) * 100
print(f"Improvement: {improvement_pct:.1f}% over baseline")

if simple_rmse < baseline_rmse:
    print("✓ Simple model beats baseline - proceed to complex models\n")
else:
    print("⚠️  Simple model doesn't beat baseline - check data quality!\n")

In [ ]:
# Step 3: Try complex model (only if simple model worked)
print("Step 3: Try Complex Model (Random Forest)")
print("=" * 50)

complex_model = rand_forest(trees=100, mode='regression')
fit_complex = complex_model.fit(formula='y ~ x1 + x2 + x3', data=train_data)
_, _, stats_complex = fit_complex.extract_outputs()
complex_rmse = stats_complex[stats_complex['metric'] == 'rmse']['value'].values[1]

print(f"Complex Model RMSE: {complex_rmse:.4f}")

improvement_vs_simple = ((simple_rmse - complex_rmse) / simple_rmse) * 100
improvement_vs_baseline = ((baseline_rmse - complex_rmse) / baseline_rmse) * 100

print(f"Improvement vs simple model: {improvement_vs_simple:.1f}%")
print(f"Improvement vs baseline:     {improvement_vs_baseline:.1f}%")

if complex_rmse < simple_rmse * 0.95:  # At least 5% better
    print("✓ Complex model significantly better - use it!")
elif complex_rmse < simple_rmse:
    print("⚠️  Complex model slightly better - maybe not worth the complexity")
else:
    print("⚠️  Complex model not better - stick with simple model!")

In [ ]:
# Step 4: Summarize
print("\n" + "=" * 50)
print("FINAL RECOMMENDATION")
print("=" * 50)

print("\nPerformance Ladder:")
results = [
    ("Baseline (null_model)", baseline_rmse),
    ("Simple (linear_reg)", simple_rmse),
    ("Complex (rand_forest)", complex_rmse)
]

for i, (name, rmse) in enumerate(sorted(results, key=lambda x: x[1]), 1):
    print(f"  {i}. {name:25s} RMSE = {rmse:.4f}")

print("\n📊 Decision Framework:")
print("  1. If NOTHING beats baseline → Don't use ML, predict mean")
print("  2. If only simple beats baseline → Use simple model")
print("  3. If complex beats simple by >5% → Use complex model")
print("  4. If complex beats simple by <5% → Stick with simple (interpretability)")

## Part 4: Extract Outputs - Three-DataFrame Structure

All fitted models in py-tidymodels return three DataFrames via `extract_outputs()`:
1. **outputs**: Predictions with actuals
2. **residuals**: Residual diagnostics
3. **stats**: Performance metrics

Let's see this for baseline models:

In [ ]:
# Extract from null model
outputs_null, residuals_null, stats_null_full = fit_null.extract_outputs()

print("DataFrame 1: outputs (predictions + actuals)")
print("=" * 50)
print(outputs_null.head(10))
print(f"\nShape: {outputs_null.shape}")
print(f"Columns: {list(outputs_null.columns)}")

In [ ]:
print("\nDataFrame 2: residuals (diagnostic info)")
print("=" * 50)
print(residuals_null.head(10))
print(f"\nShape: {residuals_null.shape}")
print(f"Columns: {list(residuals_null.columns)}")

In [ ]:
print("\nDataFrame 3: stats (performance metrics)")
print("=" * 50)
print(stats_null_full)
print(f"\nShape: {stats_null_full.shape}")
print(f"\nMetrics available: {stats_null_full['metric'].unique()}")
print(f"Splits: {stats_null_full['split'].unique()}")

In [ ]:
# Same structure for naive_reg
outputs_naive, residuals_naive, stats_naive_full = fit_naive.extract_outputs()

print("naive_reg() also returns three DataFrames:")
print("=" * 50)
print(f"outputs:   {outputs_naive.shape}")
print(f"residuals: {residuals_naive.shape}")
print(f"stats:     {stats_naive_full.shape}")
print("\nConsistent structure across ALL models in py-tidymodels!")

## Summary: Baseline Models

### null_model() - Universal Baseline

**What it does:**
- Regression: Predicts mean (or median) of training target
- Classification: Predicts mode (most frequent class)

**When to use:**
- First baseline for ANY regression/classification problem
- Reality check before complex modeling
- Quick benchmark for model selection

**Key insight:**
- RMSE of null model ≈ σ(y)
- R² = 0 by definition
- If your model has R² < 0, it's worse than null model!

### naive_reg() - Time Series Baselines

**Three methods:**

1. **Naive (Last Value)**
   - Best for: Random walks, no pattern
   - Formula: ŷ_t = y_{t-1}

2. **Seasonal Naive**
   - Best for: Strong seasonality
   - Formula: ŷ_t = y_{t-seasonal_period}

3. **Drift (Linear Trend)**
   - Best for: Consistent linear trend
   - Formula: ŷ_t = y_{t-1} + trend

**When to use:**
- Essential baseline for ANY time series forecasting
- Try all three, use the best as minimum bar
- If ML can't beat them, don't use ML!

### The Golden Rule

**If your model can't beat a simple baseline, it's not adding value.**

- Baselines are fast to implement (minutes, not hours)
- Baselines are easy to explain (everyone understands "predict the mean")
- Baselines provide minimum acceptable performance
- Complex models must justify their complexity by beating baselines

### Best Practice Workflow

1. Fit appropriate baseline (null_model or naive_reg)
2. Try simple model (e.g., linear regression)
3. Only try complex models if simple model beats baseline
4. Complex model must beat simple model by significant margin (>5%)
5. Always report performance relative to baseline

**Remember:** The best model is often the simplest one that beats the baseline!